# Прыжки

Давайте попробуем описать прыжки

In [ ]:
from first_engine.game import Game
from first_engine import objects
import pygame


class FirstGame(Game):
    def run(self):
        hero = objects.Character(self.surface, speed=8)
        hero.load_sprites(name='stand_right', path='./sprites/viking/stand/right')
        hero.load_sprites(name='stand_left', path='./sprites/viking/stand/left')
        hero.load_sprites(name='run_left', path='./sprites/viking/run/left')
        hero.load_sprites(name='run_right', path='./sprites/viking/run/right')
        
        # Создадим дополнительную преграду для тестов
        wall = objects.Barrier(self.surface, objects=[hero], width=200, height=50, x=300, y=450)

        gravity = 1 # зададим начальную скорость падения
        last_y = hero.body.y # будем отслеживать предыдущие координаты персонажа по y

        game_over = False
        while self.RUNNER:
            if not game_over:
                self.cycle_init(FPS=60)

                # Объекты
                hero.blit()
                wall.blit()

                # Движения персонажа
                hero.motion_left(sprites_active='run_left', sprite_inactive='stand_left', time_to_update=6)
                hero.motion_right(sprites_active='run_right', sprite_inactive='stand_right', time_to_update=6)
                # Роняем персонажа
                hero.body.y += gravity

                # проверяем - уперся ли персонаж в преграду
                self.window_borders([hero])
                wall.resistance()

                # проверяем что бы персонаж стоял на полу, по текущим и предыдущим координатам, 
                # а так же по силе гравитации, так как она должна действовать на персонажа, даже если он просто стоит 
                if last_y == hero.body.y and gravity != 0:
                    gravity = 1 # снижаем силу до 1
                    if pygame.key.get_pressed()[32]: # даем возможность нажать пробел
                        gravity = -20 # и если пробел был нажат, то меняем графитацию на противоположную - то есть прыжок

                if gravity < 20: # после каждой итерации меняем скорость падения - максимальная - 20
                    gravity += 1

                last_y = hero.body.y # и сохраняем положение текущей итерации для следующей

            for event in self.events():
                # print(event)  # отслеживание событий
                self.close(event)


FirstGame(width=800, height=600).run()

Мы можем видеть, что по здесь персонаж имеет 2 состояния - прижок и падение <br>
Давайте добавим функции в __Object__ и __Character__ овечающие за падение и дрыжок

In [ ]:
class Object:
    """
    Основной класс на основе которого строятся другие классы, представляющие разные типы объектов
    Содержит в себе основные методы для управление всеми объектами
    :var sprites: Dict - сохраняет спрайты созданного на его основе объекта
    :var last_action: str - информация о последнем действии объекта
    :var time_to_sprite_update: int - вряме до смены спрайта, если предполагается анимация
    :var sprite_id: int - индекс спрайта
    """
    ...
    drop_speed = 1 # добавим переменную отвечающую за скорость падения для объектов
    
    ...
    
    def drop(self, speed_up=1, max_speed=15): # добавим метод, отвечающий за падение объекта
        """
        Падение персонажа
        :return: int положение тела
        """
        self.body.y += self.drop_speed
        if self.drop_speed < max_speed:
            self.drop_speed += speed_up
        return self.drop_speed

In [ ]:
class Character(Object):
    """
    Класс для создания персонажей
    """
    def __init__(self, parent: pygame.Surface, width=40, height=40, x=0, y=0, color=(255, 255, 255),
                 speed=1, 
                 height_jump=20 # добавим аргумет отвечающий за высоту прыжка
                ):
        """
        :param parent: pygame.Surface - родимтельское окно
        :param width: int
        :param height: int
        :param x: int
        :param y: int
        :param color: tuple - цвет в RGB
        :param speed: int - скорость персонажа
        """
        ...
        self.height_jump = height_jump # сохраним высоту прыжка

    ...

    def action_jump(self): # добавим метод отвечающий за прыжок
        if self.y == self.body.y and self.drop_speed != 1:
            self.drop_speed = 1
            if pygame.key.get_pressed()[32]:
                self.drop_speed = -self.height_jump
        self.y = self.body.y

# Задание:
По практикуйтесь в создании локаций:
- Создайте платформы, пол, стены
- Загрузите фон для игры
- Попробуйте увеличить локацию за счет движения экрана